In [1]:
!pip3 install dask
!pip3 install pandas
!pip3 install pip-install-test

get all my imports

In [2]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re
import numpy as np
import time
import dask.dataframe as dd
import gzip


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


creates an artificial dataset

In [ ]:
import random
import string

num_rows = 60_000_000  #
output_file_path = "/Users/devinchau/Downloads/students_large.csv"

def random_string(length):
    return ''.join(random.choices(string.ascii_letters, k=length))

data = {
    "StudentID": [i for i in range(1, num_rows + 1)],
    "FirstName": [random_string(5) for _ in range(num_rows)],
    "LastName": [random_string(7) for _ in range(num_rows)],
    "Age": [random.randint(18, 30) for _ in range(num_rows)],
    "Gender": [random.choice(["Male", "Female", "Other"]) for _ in range(num_rows)],
    "Grade": [random.choice(["A", "B", "C", "D", "F"]) for _ in range(num_rows)],
    "Major": [random.choice(["Engineering", "Arts", "Science", "Business", "Law"]) for _ in range(num_rows)],
    "GPA": [round(random.uniform(2.0, 4.0), 2) for _ in range(num_rows)]
}

df = pd.DataFrame(data)
df.to_csv(output_file_path, index=False)


KeyboardInterrupt: 

it already ran so i did not want to redownload the values. this is for the initial step

In [3]:
file_size = os.path.getsize("/Users/devinchau/Downloads/students_large.csv")
file_size_mb = file_size / (1024 * 1024)
file_size_gb = file_size / (1024 * 1024 * 1024)
print(f"File Size: {file_size:.2f} bytes")
print(f"File Size: {file_size_mb:.2f} MB")
print(f"File Size: {file_size_gb:.2f} GB")

File Size: 2658906692.00 bytes
File Size: 2535.73 MB
File Size: 2.48 GB


In [4]:
%%writefile file.yaml
file_type: csv
dataset_name: students_large
file_name: students_large
table_name: students_large
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns:
    - StudentID
    - FirstName
    - LastName
    - Age
    - Gender
    - Major
    - GPA

Overwriting file.yaml


In [5]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime
import gc
import re




def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string)
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Overwriting testutility.py


In [38]:
import importlib.util
import sys

module_path = '/Users/devinchau/Desktop/testutility.py'

spec = importlib.util.spec_from_file_location("testutility", module_path)
testutility = importlib.util.module_from_spec(spec)
sys.modules["testutility"] = testutility  
spec.loader.exec_module(testutility)

print(testutility)


<module 'testutility' from '/Users/devinchau/Desktop/testutility.py'>


/Users/devinchau/Desktop/testutility.py:31: SyntaxWarning: invalid escape sequence '\w'
  df.columns = df.columns.str.replace('[^\w]','_',regex=True)


In [39]:
import testutility as util
config_data = util.read_config_file("file.yaml")
print(config_data)
     

{'file_type': 'csv', 'dataset_name': 'students_large', 'file_name': 'students_large', 'table_name': 'students_large', 'inbound_delimiter': ',', 'outbound_delimiter': '|', 'skip_leading_rows': 1, 'columns': ['StudentID', 'FirstName', 'LastName', 'Age', 'Gender', 'Major', 'GPA']}


In [40]:
config_data

{'file_type': 'csv',
 'dataset_name': 'students_large',
 'file_name': 'students_large',
 'table_name': 'students_large',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['StudentID',
  'FirstName',
  'LastName',
  'Age',
  'Gender',
  'Major',
  'GPA']}

reading with pandas

In [42]:
start = time.time()

pandas = pd.read_csv("/Users/devinchau/Downloads/students_large.csv",
                     delimiter = config_data['inbound_delimiter'],
                     header=0)

end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  370.4060688018799 sec


reading with dask

In [43]:
start = time.time()
dask = dd.read_csv("/Users/devinchau/Downloads/students_large.csv",
                   delimiter = config_data['inbound_delimiter'],
                   header = 0,
                   assume_missing = True)
_ = dask.head()
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  2.475955009460449 sec


basic validation

In [45]:
col_status = util.col_header_val(pandas, config_data)
if col_status:
    print("Validation successful with pandas dataframe.")
else:
    print("Validation failed with pandas dataframe.")


column name and column length validation passed
Validation successful with pandas dataframe.


writing the file in pipe separated text file (|) in gz format.

In [44]:
import gzip
import shutil

input_file = "/Users/devinchau/Downloads/students_large.csv"
output_file = "students_large.csv.gz"

with open(input_file, 'rb') as f_in:
    with gzip.open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f"File '{input_file}' compressed to '{output_file}'")
print(f"Original file size: {os.path.getsize(input_file) / (1024 * 1024):.2f} MB")
print(f"Compressed file size: {os.path.getsize(output_file) / (1024 * 1024):.2f} MB")

File '/Users/devinchau/Downloads/students_large.csv' compressed to 'students_large.csv.gz'
Original file size: 2535.73 MB
Compressed file size: 1011.12 MB
